In [1]:
import pandas as pd
import numpy as np
import os
import re
from tqdm import tqdm

In [2]:
folder_path = 'C://Users/11max/Desktop/Work/Databases/IO_Canada/'

all_data = pd.DataFrame()

for province in tqdm(['AB', 'BC', 'MB', 'NB', 'NL', 'NS', 'NT', 'NU', 'ON', 'PE', 'QC', 'SK', 'YT']):
    for year in range(2014, 2022):
        if year in [2014, 2015, 2016, 2017]:
            starting_line = 11
            starting_line_values = 16
        elif year in [2018, 2019, 2020, 2021]:
            starting_line = 3
            starting_line_values = 7
            
        use_table = pd.read_excel(os.path.join(folder_path, str(year), 'Detail level', province+'_SUT_C'+str(year)+'_D.xlsx'), 'Use_Basic')

        formatted_use_table = use_table.iloc[starting_line_values - 2:, 2:]
        formatted_use_table.index = list(zip(use_table.iloc[starting_line_values - 2:, 0].tolist(),
                            use_table.iloc[starting_line_values - 2:, 1].tolist()))
        formatted_use_table.columns = list(zip(use_table.iloc[starting_line + 1, 2:].tolist(),
                              use_table.iloc[starting_line, 2:].tolist()))
        formatted_use_table = formatted_use_table.astype(str).replace(to_replace=['.'], value='0').astype(float)
        # thousands of $ to dollars
        formatted_use_table *= 1000

        total_gfcf = pd.DataFrame([formatted_use_table.loc[[('TOTAL','Total')],
                                   [i for i in formatted_use_table.columns if (re.findall('^CO', i[0]) or 
                                                               (re.findall('^ME', i[0]) and i[1] != 'Used cars and equipment and scrap') or 
                                                               (re.findall('^IP', i[0]) and 'exports' not in i[1]))]].iloc[0]])
        total_gfcf.index = [(province, year)]

        all_data = pd.concat([all_data, total_gfcf])

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [04:34<00:00, 21.08s/it]


In [21]:
all_data.to_excel('C://Users/11max/Desktop/save.xlsx')